In [1]:
pip install praw


     -------------------------------------- 189.3/189.3 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [47]:
import os


In [43]:
import praw
from prawcore.exceptions import NotFound
import pandas as pd
from tqdm import tqdm

arr=[]
reddit = praw.Reddit(
    client_id='eLHIfVLEUxnTQlLlGt29_Q',
    client_secret='xalSQxSsLr5oxApVsrobL5dyasmCOw',
    user_agent='python:my_reddit_sentiment_app:v1.0.0 (by u/on_a_no_buy)'
)

try:
    subreddit = reddit.subreddit('sneakers')
    for post in tqdm(subreddit.hot(limit=1000)):
        #print(post.title)
        #print(post.selftext)
        if len(post.comments) > 0:
            comment=post.comments[0].body
        else:
            comment=" "
        arr.append([post.title,post.selftext,post.upvote_ratio,post.created_utc,post.score,comment])
except NotFound:
    print("The requested resource was not found. Please check the subreddit name or resource ID.")
df_sneakers = pd.DataFrame(arr, columns=['Title', 'Body','Upvote_ratio','Timestamp','Upvote_num','comments'])


951it [04:41,  3.38it/s]


In [44]:
df_sneakers

,Title,Body,Upvote_ratio,Timestamp,Upvote_num,comments
0,New Releases Thread 1/27 - 2/2,"Please post all your questions, pics and comme...",1.00,1.737954e+09,1,Nike Vomero 5 Supersonics are now available in...
1,A post about legit check posts and r/legitcheck,Hi. Please don't post legit checks here. Pleas...,0.98,1.705450e+09,141,THANK YOU! \n\nAll of the sneaker subs have be...
2,Waiting is a virtue??? Snagged these for 85€ o...,,0.95,1.738344e+09,52,https://preview.redd.it/p3pr6u7j8dge1.jpeg?wid...
3,Nike Cortez “Aloha Pack Red” (2013),,0.98,1.738339e+09,64,I'm not a huge Cortez fan but these are really...
4,Really love the details on these,,0.99,1.738286e+09,704,Just bit the bullet and grabbed a pair on Goat...
...,...,...,...,...,...,...
946,KOTD,Jeremy Fish/303 BoardsXNew Balance #390,0.71,1.737497e+09,4,
947,Hyper Royals. Don’t see these in the wild too ...,,0.84,1.737471e+09,14,Still have mine
948,I started Customizing shoes!,I’ve always been amazed by shoe customizers an...,0.79,1.737492e+09,5,Noice
949,"A couple of recent wears, 2002rs and Yeezy 700...",,0.91,1.737483e+09,8,


In [45]:
df_sneakers.to_csv('sneakers.csv', index=False)


In [48]:
# Path to the folder containing the CSV files
folder_path = 'Dataframe_new'

# List to store the DataFrames
dataframes = []

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a CSV
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV into a DataFrame
        df = pd.read_csv(file_path)
        df['topic'] = os.path.splitext(file_name)[0]
        dataframes.append(df)
        print(f"Loaded: {file_name}")

print(f"Total CSV files read: {len(dataframes)}")

Loaded: adidas.csv
Loaded: asics.csv
Loaded: fabletics.csv
Loaded: gymshark.csv
Loaded: hoka.csv
Loaded: lululemon.csv
Loaded: newbalance.csv
Loaded: nike.csv
Loaded: puma.csv
Loaded: reebok.csv
Loaded: sneakers.csv
Loaded: underarmour.csv
Total CSV files read: 12


In [49]:
combined_df = pd.concat(dataframes, ignore_index=True)


In [50]:
combined_df

,Title,Body,Upvote_ratio,Timestamp,Upvote_num,comments,topic
0,"For all the ""Can you ID"" post, just google the...",NaN,0.99,1.601858e+09,247,some reminders though\n\n* pre 2006 numbers ar...,adidas
1,Taipei-Exclusive Bubble Tea Samba,Yay or Nay?,1.00,1.738204e+09,131,That’s pretty and awesome,adidas
2,gazelle indoor - burgundy 🎈,NaN,1.00,1.738255e+09,8,"They look good, are they a new model? I swear ...",adidas
3,Team Adidas,Stan smith or Campus 80s 😉,1.00,1.738255e+09,7,Out of those 3...definitely the campus. They'r...,adidas
4,Concept kicks x Adidas ARUKU strung,NaN,0.86,1.738253e+09,5,,adidas
...,...,...,...,...,...,...,...
11453,"Hey guys, I would like you to share a personal...",NaN,1.00,1.599549e+09,11,"If you did the graphics yourself, wow. They ar...",underarmour
11454,New Project Rock Gear,"I know the PR3s just dropped yesterday, but I ...",1.00,1.599194e+09,3,9/17,underarmour
11455,Men’s Compression Shorts/Leggings with pockets...,I’ve gotten a couple pairs of the men’s compre...,1.00,1.598974e+09,3,Pretty sure there's shorts in the Project Rock...,underarmour
11456,What is the name of this bag? I found it on a ...,NaN,1.00,1.598919e+09,6,Also curious.,underarmour


In [51]:
combined_df.to_csv('athletic_apparel.csv', index=False)


In [61]:
nan_per_column = combined_df.isnull().sum()
nan_per_column

Title       0
Body     2824
topic       0
dtype: int64